In [1]:
%%capture
!pip install -q pypdf llama-index einops accelerate python-dotenv bitsandbytes -U

In [2]:
%%capture
%pip install llama-index-llms-huggingface

In [3]:
%%capture
!pip install "transformers[torch]" "huggingface_hub[inference]"

In [4]:
%%capture
!pip install llama-index-llms-llama-cpp

In [7]:
## get the data
!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

--2024-05-18 10:14:14--  https://openreview.net/pdf?id=VtmBAGCN7o
Resolving openreview.net (openreview.net)... 35.184.86.251
Connecting to openreview.net (openreview.net)|35.184.86.251|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16911937 (16M) [application/pdf]
Saving to: ‘metagpt.pdf’

metagpt.pdf         100%[===================>]  16.13M  37.5MB/s    in 0.4s    

2024-05-18 10:14:15 (37.5 MB/s) - ‘metagpt.pdf’ saved [16911937/16911937]



In [11]:
from llama_index.core import Settings, VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core import Settings
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.core.agent import ReActAgent
from llama_index.legacy.embeddings import HuggingFaceEmbedding

[nltk_data] Downloading package stopwords to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
llm = LlamaCPP(
   model_url='https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf',
   model_path=None,
   temperature=0.1,
   max_new_tokens=256,
   context_window=3900,
   generate_kwargs={},
   model_kwargs={"n_gpu_layers":-1},
   verbose=True
)

total size (MB): 4081.0


3892it [00:26, 145.37it/s]                          
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /tmp/llama_index/models/llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loa

In [9]:
Settings.llm = llm

docs = SimpleDirectoryReader(input_files=['metagpt.pdf']).load_data()

nodes = Settings.node_parser.get_nodes_from_documents(docs, show_progress=True)

print(f'len docs: {len(docs)}')
print(f'len nodes: {len(nodes)}')

Parsing nodes:   0%|          | 0/29 [00:00<?, ?it/s]

len docs: 29
len nodes: 34


In [12]:
Settings.embed_model = HuggingFaceEmbedding('thenlper/gte-large')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/670M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [13]:
index_nodes = VectorStoreIndex(nodes)

retriver_engine_nodes = index_nodes.as_retriever(similarity_top_k=3)

query_engine_nodes = index_nodes.as_query_engine(similarity_top_k=3)

In [14]:
resp = query_engine_nodes.query("Explain LLMCompiler and its usecases.")

print(resp.response)
print(resp.source_nodes)
print(resp.metadata)


llama_print_timings:        load time =  184810.89 ms
llama_print_timings:      sample time =      32.19 ms /    53 runs   (    0.61 ms per token,  1646.73 tokens per second)
llama_print_timings: prompt eval time = 1041376.32 ms /  2669 tokens (  390.17 ms per token,     2.56 tokens per second)
llama_print_timings:        eval time =   40393.70 ms /    52 runs   (  776.80 ms per token,     1.29 tokens per second)
llama_print_timings:       total time = 1081996.96 ms /  2721 tokens


 LLMCompilers are a class of AI models that can compile code written in natural language into executable code. They have various use cases such as automating software development tasks, generating code for specific programming languages, and improving the productivity of developers.
[NodeWithScore(node=TextNode(id_='02691b08-4862-4a3d-9e04-563e0aadc5c4', embedding=None, metadata={'page_label': '7', 'file_name': 'metagpt.pdf', 'file_path': 'metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2024-05-18', 'last_modified_date': '2024-05-18'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='769e7db2-2586-4ba6-9fa9-ff5d5f04878e', node_type=<ObjectType.DOCUMENT: '4'>, metadata={

In [15]:
for idx, node_with_score in enumerate(resp.source_nodes):
    print(f'Node-{idx} has the score of {node_with_score.score}')

Node-0 has the score of 0.8244340990376582
Node-1 has the score of 0.8207867540354653
Node-2 has the score of 0.8199367844755705


In [16]:
query_engine_tool = QueryEngineTool.from_defaults(
    name='llmcompiler',
    query_engine=query_engine_nodes,
    description=(
        "Provides information about LLMCompiler and Parallel Function Calling."
        ),
)

In [17]:
react_agent = ReActAgent.from_tools(
    [query_engine_tool],
    max_function_calls=10,
    llm=llm,
    verbose=True,
)

In [18]:
react_agent_resp = react_agent.chat("What is Parallel Function Calling and How LLMCompilers can help?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =  184810.89 ms
llama_print_timings:      sample time =      49.43 ms /    81 runs   (    0.61 ms per token,  1638.78 tokens per second)
llama_print_timings: prompt eval time =  188403.35 ms /   525 tokens (  358.86 ms per token,     2.79 tokens per second)
llama_print_timings:        eval time =   51629.08 ms /    80 runs   (  645.36 ms per token,     1.55 tokens per second)
llama_print_timings:       total time =  240367.14 ms /   605 tokens
Llama.generate: prefix-match hit


Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: llmcompiler
Action Input: {'input': 'What is Parallel Function Calling and How LLMCompilers can help?', 'num_beams': 5}



llama_print_timings:        load time =  184810.89 ms
llama_print_timings:      sample time =      48.76 ms /    79 runs   (    0.62 ms per token,  1620.31 tokens per second)
llama_print_timings: prompt eval time = 1419494.67 ms /  3561 tokens (  398.62 ms per token,     2.51 tokens per second)
llama_print_timings:        eval time =   64340.35 ms /    78 runs   (  824.88 ms per token,     1.21 tokens per second)
llama_print_timings:       total time = 1484180.32 ms /  3639 tokens
Llama.generate: prefix-match hit


Observation:  Parallel function calling is a technique that allows multiple functions to be executed simultaneously in different threads or processes, improving program performance by reducing execution time. LLM-based compilers can help improve parallel function calling by generating code that takes advantage of multi-core processors and other parallel computing architectures, making it possible to execute multiple functions at the same time without significant slowdowns.



llama_print_timings:        load time =  184810.89 ms
llama_print_timings:      sample time =      74.14 ms /   121 runs   (    0.61 ms per token,  1631.94 tokens per second)
llama_print_timings: prompt eval time =  237749.29 ms /   680 tokens (  349.63 ms per token,     2.86 tokens per second)
llama_print_timings:        eval time =   77680.52 ms /   120 runs   (  647.34 ms per token,     1.54 tokens per second)
llama_print_timings:       total time =  315919.56 ms /   800 tokens


Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: Parallel function calling is a technique that allows multiple functions to be executed simultaneously in different threads or processes, improving program performance by reducing execution time. LLM-based compilers can help improve parallel function calling by generating code that takes advantage of multi-core processors and other parallel computing architectures, making it possible to execute multiple functions at the same time without significant slowdowns.
user: Observation:  Thank you for your answer!
```
